DATA CLEANING AND EDA
---------------------
Objectives:- 
- Parsed datetime
- Clear decision on cancelled/diverted flights
- Chosen regression & classification targets
- A cleaned dataset saved separately

In [1]:
# Loading the parquet file
import pandas as pd
from pathlib import Path

df = pd.read_parquet("../data/processed/flights_raw.parquet")
df.shape

(7607025, 18)

In [2]:
# Objective 1:Parsing the date column to datetime format
df["fl_date"] = pd.to_datetime(df["fl_date"]) # proper datetime64 format

C:\Users\Admin\AppData\Local\Temp\ipykernel_21260\1181421557.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["fl_date"] = pd.to_datetime(df["fl_date"]) # proper datetime64 format


In [3]:
# Datetime sanity check
df["fl_date"].min(), df["fl_date"].max()


(Timestamp('2024-09-01 00:00:00'), Timestamp('2025-09-30 00:00:00'))

In [4]:
# sorting the dataframe by date
df = df.sort_values("fl_date").reset_index(drop=True)
print("Date ranges: ", df["fl_date"].min(), df["fl_date"].max()) # checking the date ranges after sorting

df.head() # checking the first few rows after sorting

Date ranges:  2024-09-01 00:00:00 2025-09-30 00:00:00


,fl_date,op_unique_carrier,origin_airport_id,origin,dest_airport_id,dest,dep_delay,dep_delay_new,arr_delay,arr_delay_new,cancelled,diverted,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,source_file
0,2024-09-01,WN,14730,SDF,12889,LAS,-13.0,0.0,-25.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,T_ONTIME_REPORTING_SEP-2024.csv
1,2024-09-01,MQ,14457,RAP,11298,DFW,-2.0,0.0,-2.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,T_ONTIME_REPORTING_SEP-2024.csv
2,2024-09-01,MQ,14457,RAP,11298,DFW,-4.0,0.0,-3.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,T_ONTIME_REPORTING_SEP-2024.csv
3,2024-09-01,MQ,14457,RAP,11298,DFW,-11.0,0.0,3.0,3.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,T_ONTIME_REPORTING_SEP-2024.csv
4,2024-09-01,MQ,14321,PWM,13930,ORD,3.0,3.0,-2.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,T_ONTIME_REPORTING_SEP-2024.csv


In [5]:
# Objective 2: Filter and drop cancelled and diverted columns
df = df[(df["cancelled"] == 0) & (df["diverted"] == 0)] # filtering out cancelled and diverted flights that have no delay info
print(df.shape)
# df.head()

df = df.drop(columns=["cancelled", "diverted"]) #"carrier_delay", "weather_delay", "nas_delay", "security_delay", "late_aircraft_delay"])
# Delay cause columns dropped as they are not needed for modeling
df.head()

(7494236, 18)


,fl_date,op_unique_carrier,origin_airport_id,origin,dest_airport_id,dest,dep_delay,dep_delay_new,arr_delay,arr_delay_new,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,source_file
0,2024-09-01,WN,14730,SDF,12889,LAS,-13.0,0.0,-25.0,0.0,NaN,NaN,NaN,NaN,NaN,T_ONTIME_REPORTING_SEP-2024.csv
1,2024-09-01,MQ,14457,RAP,11298,DFW,-2.0,0.0,-2.0,0.0,NaN,NaN,NaN,NaN,NaN,T_ONTIME_REPORTING_SEP-2024.csv
2,2024-09-01,MQ,14457,RAP,11298,DFW,-4.0,0.0,-3.0,0.0,NaN,NaN,NaN,NaN,NaN,T_ONTIME_REPORTING_SEP-2024.csv
3,2024-09-01,MQ,14457,RAP,11298,DFW,-11.0,0.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,T_ONTIME_REPORTING_SEP-2024.csv
4,2024-09-01,MQ,14321,PWM,13930,ORD,3.0,3.0,-2.0,0.0,NaN,NaN,NaN,NaN,NaN,T_ONTIME_REPORTING_SEP-2024.csv


In [6]:
# Setting the target variable for regression and classification
target_reg = "dep_delay_new"
df["is_delay"] = (df[target_reg] > 15).astype(int) # binary target for classification
# Above code answers: Was this flight delayed by more than 15 minutes? (Yes=1, No=0)
df.head()

,fl_date,op_unique_carrier,origin_airport_id,origin,dest_airport_id,dest,dep_delay,dep_delay_new,arr_delay,arr_delay_new,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,source_file,is_delay
0,2024-09-01,WN,14730,SDF,12889,LAS,-13.0,0.0,-25.0,0.0,NaN,NaN,NaN,NaN,NaN,T_ONTIME_REPORTING_SEP-2024.csv,0
1,2024-09-01,MQ,14457,RAP,11298,DFW,-2.0,0.0,-2.0,0.0,NaN,NaN,NaN,NaN,NaN,T_ONTIME_REPORTING_SEP-2024.csv,0
2,2024-09-01,MQ,14457,RAP,11298,DFW,-4.0,0.0,-3.0,0.0,NaN,NaN,NaN,NaN,NaN,T_ONTIME_REPORTING_SEP-2024.csv,0
3,2024-09-01,MQ,14457,RAP,11298,DFW,-11.0,0.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,T_ONTIME_REPORTING_SEP-2024.csv,0
4,2024-09-01,MQ,14321,PWM,13930,ORD,3.0,3.0,-2.0,0.0,NaN,NaN,NaN,NaN,NaN,T_ONTIME_REPORTING_SEP-2024.csv,0


In [7]:
# Handling missing data values in the target variable
df = df.dropna(subset=[target_reg])
df.shape

(7494236, 17)

In [8]:
# Saving the cleaned dataframe to a new parquet file
output_path = Path("../data/processed/flights_cleaned_base.parquet")
df.to_parquet(output_path, index=False)

In [10]:
new_file = pd.read_parquet("../data/processed/flights_cleaned_base.parquet")
new_file.shape
new_file.head()

,fl_date,op_unique_carrier,origin_airport_id,origin,dest_airport_id,dest,dep_delay,dep_delay_new,arr_delay,arr_delay_new,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,source_file,is_delay
0,2024-09-01,WN,14730,SDF,12889,LAS,-13.0,0.0,-25.0,0.0,NaN,NaN,NaN,NaN,NaN,T_ONTIME_REPORTING_SEP-2024.csv,0
1,2024-09-01,MQ,14457,RAP,11298,DFW,-2.0,0.0,-2.0,0.0,NaN,NaN,NaN,NaN,NaN,T_ONTIME_REPORTING_SEP-2024.csv,0
2,2024-09-01,MQ,14457,RAP,11298,DFW,-4.0,0.0,-3.0,0.0,NaN,NaN,NaN,NaN,NaN,T_ONTIME_REPORTING_SEP-2024.csv,0
3,2024-09-01,MQ,14457,RAP,11298,DFW,-11.0,0.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,T_ONTIME_REPORTING_SEP-2024.csv,0
4,2024-09-01,MQ,14321,PWM,13930,ORD,3.0,3.0,-2.0,0.0,NaN,NaN,NaN,NaN,NaN,T_ONTIME_REPORTING_SEP-2024.csv,0
